# Capstone Project - The Battle of Neighborhoods (Week 1-2)


## Business Problem section


### Background

According to the Guardian, the London Housing Market is in downturn in (in contrast to average state of British housing market) due to upcoming Brexit despite strong macroeconomic indicators such as strong employment, low mortgage rates and a lack of supply.
Business Problem
In this scenario, it is urgent to adopt machine learning tools in order to advise potential buyers in London to make effective decisions. As a result, the business problem we are currently posing is: what behavior pattern must potential buyer accept in the case of Brexit will take place?
Solution of described above problem can be found by clustering London neighborhoods in order to observe dependence between them and the average price of real estate in order to advise right venues. We will recommend venues with at least sustainable prices according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### Business problem

Business Problem In this scenario, it is urgent to adopt machine learning tools in order to advise potential buyers in London to make effective decisions. As a result, the business problem we are currently posing is: what behavior pattern must potential buyer accept in the case of Brexit will take place? Solution of described above problem can be found by clustering London neighborhoods in order to observe dependence between them and the average price of real estate in order to advise right venues. We will recommend venues with at least sustainable prices according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

## Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County. To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface.

## Methodology section

1. Collect the data about the housing market in London
2. Explore and Discover Data
3. Data preparation and preprocessing 
4. Modeling

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv")

In [3]:
df_ppd.head(5)

,{80E1AA98-0108-7BF8-E053-6C04A8C00BF2},249995,2019-01-18 00:00,SS2 6ST,T,N,F,209,Unnamed: 8,ROCHFORD ROAD,Unnamed: 10,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA.1,SOUTHEND-ON-SEA.2,A,A.1
0,{80E1AA98-010A-7BF8-E053-6C04A8C00BF2},372500,2019-01-04 00:00,RM16 2PT,S,N,F,152,NaN,LONG LANE,NaN,GRAYS,THURROCK,THURROCK,A,A
1,{80E1AA98-010C-7BF8-E053-6C04A8C00BF2},670000,2019-01-11 00:00,CM3 4BS,D,N,F,FIR TREE COTTAGE,NaN,NORTH HILL,LITTLE BADDOW,CHELMSFORD,CHELMSFORD,ESSEX,A,A
2,{80E1AA98-010D-7BF8-E053-6C04A8C00BF2},200000,2019-01-04 00:00,SS3 9RJ,T,N,F,38,NaN,EAGLE WAY,SHOEBURYNESS,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A
3,{80E1AA98-010E-7BF8-E053-6C04A8C00BF2},545000,2019-01-18 00:00,CM3 4UR,D,N,F,9,NaN,LITTLE FIELDS,DANBURY,CHELMSFORD,CHELMSFORD,ESSEX,A,A
4,{80E1AA98-010F-7BF8-E053-6C04A8C00BF2},270000,2019-01-11 00:00,SS0 9TY,T,N,F,148,NaN,WESTBOURNE GROVE,NaN,WESTCLIFF-ON-SEA,SOUTHEND-ON-SEA,SOUTHEND-ON-SEA,A,A


In [4]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [5]:

# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [6]:

df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [7]:
df_ppd_london.head(5)

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
197814,{9B361206-821B-1904-E053-6B04A8C0EEB5},1265000,2019-12-20,SW5 0NF,F,N,L,60,FLAT 1,COURTFIELD GARDENS,NaN,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A
306839,{9B361207-0321-1904-E053-6B04A8C0EEB5},1600000,2019-12-20,E17 6NQ,O,N,F,152,NaN,BLACKHORSE ROAD,NaN,LONDON,WALTHAM FOREST,GREATER LONDON,B,A
103787,{9B361206-6BFE-1904-E053-6B04A8C0EEB5},1282500,2019-12-20,N16 5UF,T,N,F,47,NaN,ALLERTON ROAD,NaN,LONDON,HACKNEY,GREATER LONDON,A,A
204521,{9B361206-8482-1904-E053-6B04A8C0EEB5},1770000,2019-12-19,N1 2EN,T,N,F,23,NaN,HALTON ROAD,NaN,LONDON,ISLINGTON,GREATER LONDON,A,A
208556,{9B361206-82C1-1904-E053-6B04A8C0EEB5},650000,2019-12-19,SW7 3BU,F,N,L,89,FLAT 5,ONSLOW GARDENS,NaN,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A


In [8]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [9]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [10]:
# Display the dataframe
df_affordable

,Street,Avg_Price
180,ALBION SQUARE,2.292500e+06
251,ALLEYN PARK,2.283095e+06
480,ARTESIAN ROAD,2.462500e+06
515,ASHCHURCH GROVE,2.425000e+06
648,AYLESBURY STREET,2.367333e+06
655,AYNHOE ROAD,2.475000e+06
953,BEDALE STREET,2.344600e+06
984,BEECHWOOD AVENUE,2.500000e+06
1132,BETTERTON STREET,2.425000e+06
1133,BETTRIDGE ROAD,2.400000e+06


In [11]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 180
item: Street       ALBION SQUARE
Avg_Price       2.2925e+06
Name: 180, dtype: object
item.Street only: ALBION SQUARE
index: 251
item: Street       ALLEYN PARK
Avg_Price     2.2831e+06
Name: 251, dtype: object
item.Street only: ALLEYN PARK
index: 480
item: Street       ARTESIAN ROAD
Avg_Price       2.4625e+06
Name: 480, dtype: object
item.Street only: ARTESIAN ROAD
index: 515
item: Street       ASHCHURCH GROVE
Avg_Price          2.425e+06
Name: 515, dtype: object
item.Street only: ASHCHURCH GROVE
index: 648
item: Street       AYLESBURY STREET
Avg_Price         2.36733e+06
Name: 648, dtype: object
item.Street only: AYLESBURY STREET
index: 655
item: Street       AYNHOE ROAD
Avg_Price      2.475e+06
Name: 655, dtype: object
item.Street only: AYNHOE ROAD
index: 953
item: Street       BEDALE STREET
Avg_Price       2.3446e+06
Name: 953, dtype: object
item.Street only: BEDALE STREET
index: 984
item: Street       BEECHWOOD AVENUE
Avg_Price             2.5e+06
Name: 984, dtype: object

In [13]:

geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:

df_affordable

,Street,Avg_Price,city_coord
180,ALBION SQUARE,2.292500e+06,"(-41.27375755, 173.28939323910353)"
251,ALLEYN PARK,2.283095e+06,"(51.4314368, -0.0857867)"
480,ARTESIAN ROAD,2.462500e+06,"(51.51562, -0.1960784)"
515,ASHCHURCH GROVE,2.425000e+06,"(51.5011208, -0.2414108)"
648,AYLESBURY STREET,2.367333e+06,"(51.5619418, -0.257793)"
655,AYNHOE ROAD,2.475000e+06,"(51.496385, -0.2169813)"
953,BEDALE STREET,2.344600e+06,"(51.5052322, -0.0896216)"
984,BEECHWOOD AVENUE,2.500000e+06,"(54.85552785, -6.290580690119052)"
1132,BETTERTON STREET,2.425000e+06,"(51.5149576, -0.1238946)"
1133,BETTRIDGE ROAD,2.400000e+06,"(51.471074, -0.2028354)"


In [16]:

df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [17]:

df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
180,ALBION SQUARE,2.292500e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
251,ALLEYN PARK,2.283095e+06,"(51.4314368, -0.0857867)",51.431437,-0.085787
480,ARTESIAN ROAD,2.462500e+06,"(51.51562, -0.1960784)",51.515620,-0.196078
515,ASHCHURCH GROVE,2.425000e+06,"(51.5011208, -0.2414108)",51.501121,-0.241411
648,AYLESBURY STREET,2.367333e+06,"(51.5619418, -0.257793)",51.561942,-0.257793
655,AYNHOE ROAD,2.475000e+06,"(51.496385, -0.2169813)",51.496385,-0.216981
953,BEDALE STREET,2.344600e+06,"(51.5052322, -0.0896216)",51.505232,-0.089622
984,BEECHWOOD AVENUE,2.500000e+06,"(54.85552785, -6.290580690119052)",54.855528,-6.290581
1132,BETTERTON STREET,2.425000e+06,"(51.5149576, -0.1238946)",51.514958,-0.123895
1133,BETTRIDGE ROAD,2.400000e+06,"(51.471074, -0.2028354)",51.471074,-0.202835


In [18]:

address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [19]:

df = df_affordable.drop(columns=['city_coord'])

In [20]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [21]:

#Define Foursquare Credentials and Version

CLIENT_ID = 'EY3ULKHYS1T2ERNER2GWKXJUWIGHK0UZ4LSZ0D35LVDJTACE' # Foursquare ID
CLIENT_SECRET = 'TDMEPW20PUF5YPU45Q3ATHF0W0VR1EYYQFG41DU1ELB21NBI' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EY3ULKHYS1T2ERNER2GWKXJUWIGHK0UZ4LSZ0D35LVDJTACE
CLIENT_SECRET:TDMEPW20PUF5YPU45Q3ATHF0W0VR1EYYQFG41DU1ELB21NBI


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ALLEYN PARK
ARTESIAN ROAD
ASHCHURCH GROVE
AYLESBURY STREET
AYNHOE ROAD
BEDALE STREET
BEECHWOOD AVENUE
BETTERTON STREET
BETTRIDGE ROAD
BILLING ROAD
BOURDON STREET
BRADFIELD ROAD
BRAMHAM GARDENS
BROMPTON PLACE
BRUNSWICK PLACE
CABBELL STREET
CAITHNESS ROAD
CAMBRIDGE TERRACE
CAMDEN HIGH STREET
CAMDEN SQUARE
CAMPDEN HILL ROAD
CANONBURY PARK SOUTH
CARLISLE PLACE
CARLTON GARDENS
CHELSEA EMBANKMENT
CHENISTON GARDENS
CHESHAM MEWS
COLERIDGE ROAD
COLLINGHAM ROAD
COULTER ROAD
COURT LANE GARDENS
COURTNELL STREET
CREDITON HILL
CROFTDOWN ROAD
DARTMOUTH PARK AVENUE
DE VERE GARDENS
DEEPDALE
DEER PARK ROAD
DEVEREUX LANE
DEVONSHIRE MEWS WEST
DOVER STREET
DOWNSIDE CRESCENT
DUDLEY ROAD
DUKES LANE
EATON TERRACE MEWS
ELLERBY STREET
ELVASTON PLACE
ENNISMORE GARDENS MEWS
ESSEX STREET
EYNELLA ROAD
FENCHURCH STREET
FOUNTAYNE ROAD
FOURNIER STREET
FREWIN ROAD
FRISTON STREET
GLOUCESTER PLACE MEWS
GORDON COTTAGES
GORST ROAD
GRAHAM TERRACE
GUTHRIE STREET
HALLAM STREET
HALSEY STREET
HAMPSTEAD LANE
HANS C

In [24]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
4,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
5,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
6,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
7,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
8,ALBION SQUARE,-41.273758,173.289393,The Kitchen,-41.272360,173.285500,Café
9,ALBION SQUARE,-41.273758,173.289393,The Vic Mac's Brew Bar,-41.274757,173.283914,Pub


In [25]:

location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ALLEYN PARK,8,8,8,8,8,8
ARTESIAN ROAD,100,100,100,100,100,100
ASHCHURCH GROVE,28,28,28,28,28,28
AYLESBURY STREET,6,6,6,6,6,6
AYNHOE ROAD,34,34,34,34,34,34
BEDALE STREET,100,100,100,100,100,100
BEECHWOOD AVENUE,2,2,2,2,2,2
BETTERTON STREET,100,100,100,100,100,100


In [26]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 306 uniques categories.


In [27]:
location_venues.shape

(4426, 7)

In [28]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,African Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,African Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,ALBION SQUARE,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.037037,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0
1,ALLEYN PARK,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0
2,ARTESIAN ROAD,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,...,0.000000,0.0,0.000000,0.010000,0.000000,0.00,0.000000,0.0,0.000000,0.0
3,ASHCHURCH GROVE,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.035714,0.00,0.000000,0.0,0.000000,0.0
4,AYLESBURY STREET,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0
5,AYNHOE ROAD,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0
6,BEDALE STREET,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.040000,0.000000,0.00,0.000000,0.0,0.000000,0.0
7,BEECHWOOD AVENUE,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0
8,BETTERTON STREET,0.02,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,...,0.000000,0.0,0.000000,0.030000,0.000000,0.01,0.010000,0.0,0.000000,0.0
9,BETTRIDGE ROAD,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.024390,0.000000,0.00,0.000000,0.0,0.048780,0.0


In [30]:
london_grouped.shape

(126, 307)

In [31]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.22
1  Indian Restaurant  0.07
2                Pub  0.07
3         Restaurant  0.07
4        Coffee Shop  0.07


----ALLEYN PARK----
                venue  freq
0             Brewery  0.25
1                Café  0.25
2                 Pub  0.12
3  Italian Restaurant  0.12
4       Train Station  0.12


----ARTESIAN ROAD----
               venue  freq
0              Hotel  0.07
1                Pub  0.06
2               Café  0.05
3             Garden  0.04
4  Indian Restaurant  0.03


----ASHCHURCH GROVE----
                      venue  freq
0                       Pub  0.14
1             Grocery Store  0.14
2                    Bakery  0.07
3         Indian Restaurant  0.07
4  Mediterranean Restaurant  0.07


----AYLESBURY STREET----
                  venue  freq
0         Grocery Store  0.50
1  Fast Food Restaurant  0.17
2           Coffee Shop  0.17
3           Supermarket  0.17
4     Accessories Store  0.00


-



----EATON TERRACE MEWS----
                venue  freq
0  Italian Restaurant  0.08
1                Café  0.08
2               Hotel  0.07
3          Restaurant  0.06
4              Bakery  0.05


----ELLERBY STREET----
            venue  freq
0            Park  0.17
1   Grocery Store  0.17
2        Pharmacy  0.17
3          Bakery  0.17
4  Sandwich Place  0.17


----ELVASTON PLACE----
                      venue  freq
0                     Hotel  0.15
1            Science Museum  0.09
2                   Exhibit  0.07
3               Coffee Shop  0.06
4  Mediterranean Restaurant  0.04


----ENNISMORE GARDENS MEWS----
                venue  freq
0                Café  0.16
1  Italian Restaurant  0.09
2             Exhibit  0.07
3      Science Museum  0.07
4               Hotel  0.07


----ESSEX STREET----
                 venue  freq
0           Food Truck  0.11
1                  Gym  0.06
2  American Restaurant  0.06
3                 Park  0.06
4       Sandwich Place  0.06


----E

                venue  freq
0                 Pub  0.12
1                Café  0.10
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4       Historic Site  0.04


----PARKE ROAD----
               venue  freq
0                Pub  0.50
1               Park  0.25
2              River  0.25
3  Accessories Store  0.00
4       Optical Shop  0.00


----PARR'S WAY----
                   venue  freq
0             Playground  0.25
1  General Entertainment  0.25
2         Discount Store  0.25
3           Cocktail Bar  0.25
4        Organic Grocery  0.00


----PARSONS GATE MEWS----
                venue  freq
0         Coffee Shop  0.08
1                 Pub  0.08
2       Grocery Store  0.08
3  Italian Restaurant  0.06
4         Yoga Studio  0.06


----PAULTONS SQUARE----
                venue  freq
0  Italian Restaurant  0.09
1                Café  0.09
2                 Pub  0.04
3    Asian Restaurant  0.04
4           Nightclub  0.04


----PAVILION ROAD----
               venue  freq
0

In [32]:

# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [34]:

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [35]:

venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Restaurant,Pub,Indian Restaurant,Coffee Shop,Bar,French Restaurant,Seafood Restaurant,Beer Garden,Museum
1,ALLEYN PARK,Brewery,Café,Indian Restaurant,Train Station,Pub,Italian Restaurant,Indoor Play Area,Fast Food Restaurant,Event Space,Exhibit
2,ARTESIAN ROAD,Hotel,Pub,Café,Garden,Persian Restaurant,Indian Restaurant,Restaurant,Gym / Fitness Center,Coffee Shop,Greek Restaurant
3,ASHCHURCH GROVE,Pub,Grocery Store,Mediterranean Restaurant,Indian Restaurant,Coffee Shop,Bakery,Fish & Chips Shop,Park,Tea Room,Thai Restaurant
4,AYLESBURY STREET,Grocery Store,Coffee Shop,Supermarket,Fast Food Restaurant,Zoo,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [36]:

venues_sorted.shape

(126, 11)

In [37]:
london_grouped.shape

(126, 307)

In [38]:

london_grouped=df

In [39]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 4, 3, 3, 2, 0, 2, 0, 3, 2, 2, 1, 1, 1, 2, 3, 4, 1, 3, 0, 3, 4,
       2, 3, 0, 1, 4, 4, 2, 0, 2, 3, 4, 4, 0, 2, 1, 0, 3, 4, 2, 0, 2, 4,
       2, 2, 3, 2, 1, 3], dtype=int32)

In [40]:

#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
180,ALBION SQUARE,2.292500e+06,-41.273758,173.289393
251,ALLEYN PARK,2.283095e+06,51.431437,-0.085787
480,ARTESIAN ROAD,2.462500e+06,51.515620,-0.196078
515,ASHCHURCH GROVE,2.425000e+06,51.501121,-0.241411
648,AYLESBURY STREET,2.367333e+06,51.561942,-0.257793


In [41]:

london_grouped_clustering.shape

(138, 4)

In [42]:

df.shape

(138, 4)

In [43]:

london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [44]:

df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [45]:

# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
180,ALBION SQUARE,2.292500e+06,-41.273758,173.289393,4,Café,Restaurant,Pub,Indian Restaurant,Coffee Shop,Bar,French Restaurant,Seafood Restaurant,Beer Garden,Museum
251,ALLEYN PARK,2.283095e+06,51.431437,-0.085787,4,Brewery,Café,Indian Restaurant,Train Station,Pub,Italian Restaurant,Indoor Play Area,Fast Food Restaurant,Event Space,Exhibit
480,ARTESIAN ROAD,2.462500e+06,51.515620,-0.196078,3,Hotel,Pub,Café,Garden,Persian Restaurant,Indian Restaurant,Restaurant,Gym / Fitness Center,Coffee Shop,Greek Restaurant
515,ASHCHURCH GROVE,2.425000e+06,51.501121,-0.241411,3,Pub,Grocery Store,Mediterranean Restaurant,Indian Restaurant,Coffee Shop,Bakery,Fish & Chips Shop,Park,Tea Room,Thai Restaurant
648,AYLESBURY STREET,2.367333e+06,51.561942,-0.257793,2,Grocery Store,Coffee Shop,Supermarket,Fast Food Restaurant,Zoo,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
655,AYNHOE ROAD,2.475000e+06,51.496385,-0.216981,0,Pub,Hotel,Gastropub,Coffee Shop,Park,Cocktail Bar,Italian Restaurant,Steakhouse,Farmers Market,Sandwich Place
953,BEDALE STREET,2.344600e+06,51.505232,-0.089622,2,Coffee Shop,Pub,Wine Bar,Italian Restaurant,Restaurant,Hotel,Seafood Restaurant,Burger Joint,Portuguese Restaurant,Food Truck
984,BEECHWOOD AVENUE,2.500000e+06,54.855528,-6.290581,0,Performing Arts Venue,Restaurant,Zoo,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
1132,BETTERTON STREET,2.425000e+06,51.514958,-0.123895,3,Theater,Coffee Shop,Clothing Store,Wine Bar,Cosmetics Shop,Dessert Shop,Gym,Indian Restaurant,Café,Hotel
1133,BETTRIDGE ROAD,2.400000e+06,51.471074,-0.202835,2,Pub,Grocery Store,Italian Restaurant,Coffee Shop,Café,Park,French Restaurant,Bakery,Yoga Studio,Food & Drink Shop


In [46]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
655,2475000.0,Pub,Hotel,Gastropub,Coffee Shop,Park,Cocktail Bar,Italian Restaurant,Steakhouse,Farmers Market,Sandwich Place
984,2500000.0,Performing Arts Venue,Restaurant,Zoo,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
2030,2494660.0,Coffee Shop,Pub,Bar,Market,Vegetarian / Vegan Restaurant,Music Venue,Burger Joint,Café,Caribbean Restaurant,Clothing Store
2140,2500000.0,Italian Restaurant,Café,Dessert Shop,Hotel,Pizza Place,Convenience Store,Coffee Shop,Park,Lebanese Restaurant,Deli / Bodega
2854,2477000.0,Hotel,Pub,Coffee Shop,Chinese Restaurant,Garden,Café,Italian Restaurant,Pizza Place,Thai Restaurant,Cocktail Bar


In [48]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1385,2.200000e+06,Boutique,Clothing Store,Art Gallery,Italian Restaurant,Cocktail Bar,Lounge,Indian Restaurant,French Restaurant,Hotel,Café
1448,2.225000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,2.235207e+06,Hotel,Garden,Café,Pub,Indian Restaurant,Pizza Place,Italian Restaurant,Chinese Restaurant,Tapas Restaurant,Coffee Shop
1970,2.218333e+06,Fish & Chips Shop,Bus Stop,Grocery Store,Fried Chicken Joint,Food Court,Food & Drink Shop,Food Stand,Food,Flea Market,Fish Market
2448,2.213750e+06,Pub,French Restaurant,Garden,Harbor / Marina,Plaza,Pizza Place,Park,Outdoor Sculpture,Museum,Monument / Landmark


In [49]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
648,2.367333e+06,Grocery Store,Coffee Shop,Supermarket,Fast Food Restaurant,Zoo,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
953,2.344600e+06,Coffee Shop,Pub,Wine Bar,Italian Restaurant,Restaurant,Hotel,Seafood Restaurant,Burger Joint,Portuguese Restaurant,Food Truck
1133,2.400000e+06,Pub,Grocery Store,Italian Restaurant,Coffee Shop,Café,Park,French Restaurant,Bakery,Yoga Studio,Food & Drink Shop
1173,2.400000e+06,Pub,Soccer Stadium,Restaurant,Coffee Shop,Grocery Store,Park,Sports Bar,Italian Restaurant,Sandwich Place,Café
1676,2.400000e+06,Pizza Place,Lounge,Convenience Store,Sandwich Place,Business Service,Fast Food Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm


In [50]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
480,2462500.0,Hotel,Pub,Café,Garden,Persian Restaurant,Indian Restaurant,Restaurant,Gym / Fitness Center,Coffee Shop,Greek Restaurant
515,2425000.0,Pub,Grocery Store,Mediterranean Restaurant,Indian Restaurant,Coffee Shop,Bakery,Fish & Chips Shop,Park,Tea Room,Thai Restaurant
1132,2425000.0,Theater,Coffee Shop,Clothing Store,Wine Bar,Cosmetics Shop,Dessert Shop,Gym,Indian Restaurant,Café,Hotel
1775,2455000.0,Grocery Store,Convenience Store,Gas Station,Zoo,Filipino Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
2028,2420000.0,Shopping Mall,Convenience Store,Pharmacy,Zoo,Fast Food Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [51]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
180,2.292500e+06,Café,Restaurant,Pub,Indian Restaurant,Coffee Shop,Bar,French Restaurant,Seafood Restaurant,Beer Garden,Museum
251,2.283095e+06,Brewery,Café,Indian Restaurant,Train Station,Pub,Italian Restaurant,Indoor Play Area,Fast Food Restaurant,Event Space,Exhibit
1942,2.300000e+06,Middle Eastern Restaurant,Coffee Shop,Hotel,Pub,Sandwich Place,Gym / Fitness Center,Bookstore,Pharmacy,Japanese Restaurant,Café
2048,2.305400e+06,Café,Clothing Store,Italian Restaurant,Restaurant,Burger Joint,Grocery Store,Modern European Restaurant,Gym / Fitness Center,Bakery,Juice Bar
2463,2.310000e+06,Café,Pub,Hotel,Clothing Store,Italian Restaurant,French Restaurant,Burger Joint,Bakery,Modern European Restaurant,English Restaurant


## Results and Discussion section


First of all the London Housing market is in downturn , it is still atractive because of huge number of elite venues.
First, is is worth doing to examine under-priced venues such as South-West London (Wandsworth, Balham) and North-West London (Isliington) looks as very interesting with   next future elite venues with a wide range of amenities and facilities.
In terms of suistainability West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, 
Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 3, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 4, may target individuals who love pubs, Café and Restaurant	.

## Conclusion

In te event of Brexit, it is urgent to adopt machine learning tools in order to assist potential house buyers in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?
To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.
First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.
Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.
Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. Accordingly, we discovered the patterns which has shown The first pattern we are referring to, i.e. Clusters 0, 2 and 3, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 4, may target individuals who love pubs, Café and Restaurant. 